**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2025*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.ticker as mticker

import seaborn
seaborn.set_context('talk')

from sqlalchemy import create_engine, text

In [2]:
import plotly
plotly.__version__


'6.1.0'

In [3]:
# # data source: Melbourne 
melb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')

# data source: Airbnb-Melbourne
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols)

/var/folders/9x/sk5wzh9n465bsgp0vntddj3w0000gn/T/ipykernel_37116/3089177150.py:13: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_df = pd.read_csv(


## Ejercicio 1 SQL:

1. Crear una base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Validar tipos de columnas antes de guardar: df.to_sql() infiere tipos automáticamente, pero a veces los nombres de columnas como Date, Price o ID dan problemas si vienen con strings o valores nulos.

4. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.
    - Consulta con filtro: ¿Cuántas propiedades hay por ciudad con más de 2 habitaciones?
    - Agregación condicional: ¿Cuál es el precio promedio de propiedades según tipo (Type) y ciudad?
    - Orden y límites: Mostrá el top 5 barrios con propiedades más caras en promedio.

5. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  



### Pequena exploracion de datos de las bases **melf_df**, **airnbnb_df**

Realizamos una pequena exploracion de los dataframes *melf_df* y *airbnb_df*, y entendemos que las bases de datos, mismo que puedan ser relacionadas, se encuentran con diferentes niveles de informaciones. La base *melf_df* es una base con nivel de informacion de precios de ventas de inmuebles de diversos barrios en la ciudad de Melbourne, Australia (entendemos que el campo 'suburb' puede ser considerado el barrio segun la siguiente pagina de referencia de https://www.melbourne.vic.gov.au/suburbs-and-neighbourhoods), mientras que la base *airbnb_df* tiene informacion de alquiler de diarios/semanal/mensual de diferentes ciudades Australia, incluyendo Melbourne. Tambien consideramos y entendemos que cada registro de la base puede tener un 'unique_id' para cada inmueble que puede ser considerado el campo 'Postcode', mientras que en la base *airbnb_df* ese campo seria 'zipcode', porque lo que podemos conectar ambas las fuentes considerando mediante estos campos. Pero como en algunas de las preguntas necesitamos responder y dar informaciones sobre cantidade de habitaciones, precio promedio de las propiedades o las propiedades mas caras, si realizamos un JOIN entre las bases vamos a terminando con informacion solamente para la ciudad de Melbourne y esto es debido a que la base airbnb generada en clase no tiene la informacion de la cantidad de habitaciones, tipo, precio de venta de inmueble, por esta razon consideramos trabajar directamente con la base *melf_df*, para responder las preguntas del apartado 3.


**Nota:** Realizamos una pequena exploracion de otros columnas que estan presentes en la base de datos 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv' y podemos ver que tiene informacion complementar que no fue considera en la clase como por ejemplo la cantidad de habitaciones, banos, etc. Por exemplo: 

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighborhood', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode','smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'requires_license', 'license', 'instant_bookable', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification', 'calculated_host_listings_count', 'reviews_per_month']


Creacion de Base de datos e ingesta

In [4]:
# Create a SQLite database -- engine

engine = create_engine("sqlite:///melbourne.db", echo=True)

# Working data for the analysis
airbnb_df['zipcode'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce')
airbnb_df['zipcode_int'] = airbnb_df.zipcode.fillna(0).astype('int') # Convertimos la columna zipcode a int y renombramos la columna para no perder la original
melb_df['Postcode'] = pd.to_numeric(melb_df.Postcode, errors='coerce')
melb_df['Postcode_int'] = melb_df.Postcode.fillna(0).astype('int') # Convertimos la columna Postcode a int y renombramos la columna para no perder la original
airbnb_df['weekly_price_calculate'] = airbnb_df.apply(lambda row: row['price']*7. if pd.isnull(row['weekly_price']) else row['weekly_price'], axis=1) # Realizamos el calculo de weekly_price considerando price*7dias
airbnb_df['monthly_price_calculate'] = airbnb_df.apply(lambda row: row['price']*30. if pd.isnull(row['monthly_price']) else row['monthly_price'], axis=1) # Realizamos el calculo de weekly_price considerando price*30dias
melb_df['Date'] = pd.to_datetime(melb_df['Date'], errors='coerce') # Convertimos la columna Date a datetime
melb_df['Bedroom2'] = melb_df['Bedroom2'].fillna(0).astype('int') # Convertimos la columna Bedroom2 a int
melb_df['Bathroom'] = melb_df['Bathroom'].fillna(0).astype('int') # Convertimos la columna Bathroom a int
melb_df['Car'] = melb_df['Car'].fillna(0).astype('int') # Convertimos la columna Car a int


In [53]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  object        
 1   Address        13580 non-null  object        
 2   Rooms          13580 non-null  int64         
 3   Type           13580 non-null  object        
 4   Price          13580 non-null  float64       
 5   Method         13580 non-null  object        
 6   SellerG        13580 non-null  object        
 7   Date           5138 non-null   datetime64[ns]
 8   Distance       13580 non-null  float64       
 9   Postcode       13580 non-null  float64       
 10  Bedroom2       13580 non-null  int64         
 11  Bathroom       13580 non-null  int64         
 12  Car            13580 non-null  int64         
 13  Landsize       13580 non-null  float64       
 14  BuildingArea   7130 non-null   float64       
 15  YearBuilt      8205

In [54]:
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22895 entries, 0 to 22894
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   description              22563 non-null  object 
 1   neighborhood_overview    14424 non-null  object 
 2   street                   22895 non-null  object 
 3   neighborhood             17082 non-null  object 
 4   city                     22895 non-null  object 
 5   suburb                   22872 non-null  object 
 6   state                    22834 non-null  object 
 7   zipcode                  22749 non-null  float64
 8   latitude                 22895 non-null  float64
 9   longitude                22895 non-null  float64
 10  price                    22895 non-null  int64  
 11  weekly_price             2524 non-null   float64
 12  monthly_price            1891 non-null   float64
 13  zipcode_int              22895 non-null  int64  
 14  weekly_price_calculate

In [55]:
# Create a table 'melb' in the SQLite database
melb_df.to_sql('melb', con=engine, if_exists='replace', index=False)

2025-05-20 20:59:40,047 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:40,104 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("melb")
2025-05-20 20:59:40,105 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:40,108 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("melb")
2025-05-20 20:59:40,108 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:40,123 INFO sqlalchemy.engine.Engine 
CREATE TABLE melb (
	"Suburb" TEXT, 
	"Address" TEXT, 
	"Rooms" BIGINT, 
	"Type" TEXT, 
	"Price" FLOAT, 
	"Method" TEXT, 
	"SellerG" TEXT, 
	"Date" DATETIME, 
	"Distance" FLOAT, 
	"Postcode" FLOAT, 
	"Bedroom2" BIGINT, 
	"Bathroom" BIGINT, 
	"Car" BIGINT, 
	"Landsize" FLOAT, 
	"BuildingArea" FLOAT, 
	"YearBuilt" FLOAT, 
	"CouncilArea" TEXT, 
	"Lattitude" FLOAT, 
	"Longtitude" FLOAT, 
	"Regionname" TEXT, 
	"Propertycount" FLOAT, 
	"Postcode_int" BIGINT
)


2025-05-20 20:59:40,124 INFO sqlalchemy.engine.Engine [no key 0.00165s] ()
2025-05-20 20:59:40,381 INFO sqlalc

13580

In [56]:
# Create a table 'airbnb' in the SQLite database and insert the data
airbnb_df.to_sql('airbnb', con=engine, if_exists='replace', index=False)

2025-05-20 20:59:41,317 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:41,329 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("airbnb")
2025-05-20 20:59:41,331 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:41,332 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("airbnb")
2025-05-20 20:59:41,332 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:41,335 INFO sqlalchemy.engine.Engine 
CREATE TABLE airbnb (
	description TEXT, 
	neighborhood_overview TEXT, 
	street TEXT, 
	neighborhood TEXT, 
	city TEXT, 
	suburb TEXT, 
	state TEXT, 
	zipcode FLOAT, 
	latitude FLOAT, 
	longitude FLOAT, 
	price BIGINT, 
	weekly_price FLOAT, 
	monthly_price FLOAT, 
	zipcode_int BIGINT, 
	weekly_price_calculate FLOAT, 
	monthly_price_calculate FLOAT
)


2025-05-20 20:59:41,336 INFO sqlalchemy.engine.Engine [no key 0.00074s] ()
2025-05-20 20:59:41,656 INFO sqlalchemy.engine.Engine INSERT INTO airbnb (description, neighborhood_overview, street, neighborhood, city,

22895

Vimos que los datos cuando fueron guardados en la base de datos SQLite usando a estrutura interna de Pandas, algunos campos como 'Date', 'Bathroom2', 'Car', 'Bathroom' fueron en formatos que no fueron especificados. Por esa realizamos algunas modificaciones previamente en esos campos para poder guardarlos en un formato mas apto para cada campo. Tambien, pudimos observar en la documentacion de pandas que existe una parametro (dtype={'name_column1':Integer(), 'name_column2':Float(),.....}) que puede ser pasado en el momento de guardar los datos especificando los formatos de cada campo.

In [57]:
# Para saber el schema de la tabla melb
diff_schema_melb= pd.read_sql(text("""PRAGMA table_info('melb')"""), con=engine)
diff_schema_melb['pandas_column_type'] = melb_df.dtypes
diff_schema_melb[['name','type','pandas_column_type']]

2025-05-20 20:59:43,902 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:43,903 INFO sqlalchemy.engine.Engine PRAGMA table_info('melb')
2025-05-20 20:59:43,904 INFO sqlalchemy.engine.Engine [generated in 0.00221s] ()
2025-05-20 20:59:43,909 INFO sqlalchemy.engine.Engine ROLLBACK


,name,type,pandas_column_type
0,Suburb,TEXT,NaN
1,Address,TEXT,NaN
2,Rooms,BIGINT,NaN
3,Type,TEXT,NaN
4,Price,FLOAT,NaN
5,Method,TEXT,NaN
6,SellerG,TEXT,NaN
7,Date,DATETIME,NaN
8,Distance,FLOAT,NaN
9,Postcode,FLOAT,NaN


In [58]:
# Para saber el schema de la tabla airbnb
pd.read_sql(text("""PRAGMA table_info('airbnb')"""), con=engine)

2025-05-20 20:59:44,817 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:44,818 INFO sqlalchemy.engine.Engine PRAGMA table_info('airbnb')
2025-05-20 20:59:44,819 INFO sqlalchemy.engine.Engine [generated in 0.00227s] ()
2025-05-20 20:59:44,823 INFO sqlalchemy.engine.Engine ROLLBACK


,cid,name,type,notnull,dflt_value,pk
0,0,description,TEXT,0,None,0
1,1,neighborhood_overview,TEXT,0,None,0
2,2,street,TEXT,0,None,0
3,3,neighborhood,TEXT,0,None,0
4,4,city,TEXT,0,None,0
5,5,suburb,TEXT,0,None,0
6,6,state,TEXT,0,None,0
7,7,zipcode,FLOAT,0,None,0
8,8,latitude,FLOAT,0,None,0
9,9,longitude,FLOAT,0,None,0


In [59]:
queryTotalRegistrosPorCiudad = "select count(*) as total from melb"
resultTotalRegistrosPorCiudad = pd.read_sql(queryTotalRegistrosPorCiudad, con=engine)
resultTotalRegistrosPorCiudad

2025-05-20 20:59:46,607 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:46,608 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("select count(*) as total from melb")
2025-05-20 20:59:46,609 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:46,610 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("select count(*) as total from melb")
2025-05-20 20:59:46,610 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:46,611 INFO sqlalchemy.engine.Engine select count(*) as total from melb
2025-05-20 20:59:46,611 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:46,616 INFO sqlalchemy.engine.Engine ROLLBACK


,total
0,13580


In [60]:
queryTotalRegistrosPorBarrioYCiudad = "select count(*) as total, suburb from melb group by suburb order by total desc;"
resultTotalRegistrosPorBarrioYCiudad = pd.read_sql(queryTotalRegistrosPorBarrioYCiudad, con=engine)
resultTotalRegistrosPorBarrioYCiudad

2025-05-20 20:59:47,165 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:47,166 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("select count(*) as total, suburb from melb group by suburb order by total desc;")
2025-05-20 20:59:47,167 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:47,168 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("select count(*) as total, suburb from melb group by suburb order by total desc;")
2025-05-20 20:59:47,169 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:47,169 INFO sqlalchemy.engine.Engine select count(*) as total, suburb from melb group by suburb order by total desc;
2025-05-20 20:59:47,170 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:47,178 INFO sqlalchemy.engine.Engine ROLLBACK


,total,Suburb
0,359,Reservoir
1,260,Richmond
2,249,Bentleigh East
3,239,Preston
4,222,Brunswick
...,...,...
309,1,Bullengarook
310,1,Brookfield
311,1,Beaconsfield Upper
312,1,Bacchus Marsh


In [61]:
    # - Consulta con filtro: ¿Cuántas propiedades hay por ciudad con más de 2 habitaciones?
    
queryTotalRegistrosPorCiudadConMasDe2Habitaciones = "select count(*) as total from melb where rooms > 2 order by total desc;"
resultTotalRegistrosPorCiudadConMasDe2Habitaciones = pd.read_sql(queryTotalRegistrosPorCiudadConMasDe2Habitaciones, con=engine)
resultTotalRegistrosPorCiudadConMasDe2Habitaciones


2025-05-20 20:59:47,696 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:47,698 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("select count(*) as total from melb where rooms > 2 order by total desc;")
2025-05-20 20:59:47,698 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:47,699 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("select count(*) as total from melb where rooms > 2 order by total desc;")
2025-05-20 20:59:47,700 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:47,700 INFO sqlalchemy.engine.Engine select count(*) as total from melb where rooms > 2 order by total desc;
2025-05-20 20:59:47,701 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:47,704 INFO sqlalchemy.engine.Engine ROLLBACK


,total
0,9251


In [62]:
    # - Agregación condicional: ¿Cuál es el precio promedio de propiedades según tipo (Type) y ciudad?
queryPrecioPromedioPorTipoYCiudad = "select round(avg(price) ,2) as precio_promedio, type  from melb group by type order by precio_promedio desc;"
resultPrecioPromedioPorTipoYCiudad = pd.read_sql(queryPrecioPromedioPorTipoYCiudad, con=engine)
resultPrecioPromedioPorTipoYCiudad

2025-05-20 20:59:48,300 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:48,302 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("select round(avg(price) ,2) as precio_promedio, type  from melb group by type order by precio_promedio desc;")
2025-05-20 20:59:48,302 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:48,303 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("select round(avg(price) ,2) as precio_promedio, type  from melb group by type order by precio_promedio desc;")
2025-05-20 20:59:48,303 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:48,304 INFO sqlalchemy.engine.Engine select round(avg(price) ,2) as precio_promedio, type  from melb group by type order by precio_promedio desc;
2025-05-20 20:59:48,304 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:48,311 INFO sqlalchemy.engine.Engine ROLLBACK


,precio_promedio,Type
0,1242664.76,h
1,933735.05,t
2,605127.48,u


In [63]:
    # - Orden y límites: Mostrá el top 5 barrios con propiedades más caras en promedio.
queryTop5BarriosConPropiedadesMasCaras = "select round(avg(price) ,2) as precio_promedio, suburb from melb group by suburb order by precio_promedio desc limit 5;"
resultTop5BarriosConPropiedadesMasCaras = pd.read_sql(queryTop5BarriosConPropiedadesMasCaras, con=engine)
resultTop5BarriosConPropiedadesMasCaras


2025-05-20 20:59:48,864 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:48,865 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("select round(avg(price) ,2) as precio_promedio, suburb from melb group by suburb order by precio_promedio desc limit 5;")
2025-05-20 20:59:48,866 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:48,867 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("select round(avg(price) ,2) as precio_promedio, suburb from melb group by suburb order by precio_promedio desc limit 5;")
2025-05-20 20:59:48,867 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:48,868 INFO sqlalchemy.engine.Engine select round(avg(price) ,2) as precio_promedio, suburb from melb group by suburb order by precio_promedio desc limit 5;
2025-05-20 20:59:48,868 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:48,881 INFO sqlalchemy.engine.Engine ROLLBACK


,precio_promedio,Suburb
0,2185000.00,Kooyong
1,2180240.74,Canterbury
2,2082529.41,Middle Park
3,1941355.07,Albert Park
4,1930158.00,Brighton


Ejercicio 5

In [64]:
queryJoin = """
select m.*, a.*
from melb m 
left join airbnb a 
    on m.Postcode_int = a.zipcode_int
LIMIT 100
"""
resultJoin = pd.read_sql(queryJoin, con=engine)
resultJoin

2025-05-20 20:59:53,583 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 20:59:53,584 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
select m.*, a.*
from melb m 
left join airbnb a 
    on m.Postcode_int = a.zipcode_int
LIMIT 100
")
2025-05-20 20:59:53,585 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:53,587 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
select m.*, a.*
from melb m 
left join airbnb a 
    on m.Postcode_int = a.zipcode_int
LIMIT 100
")
2025-05-20 20:59:53,587 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:53,588 INFO sqlalchemy.engine.Engine 
select m.*, a.*
from melb m 
left join airbnb a 
    on m.Postcode_int = a.zipcode_int
LIMIT 100

2025-05-20 20:59:53,589 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 20:59:53,997 INFO sqlalchemy.engine.Engine ROLLBACK


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,state,zipcode,latitude,longitude,price,weekly_price,monthly_price,zipcode_int,weekly_price_calculate,monthly_price_calculate
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.810196,145.008976,125,NaN,NaN,3067,875.0,3750.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.810981,145.014269,96,NaN,NaN,3067,672.0,2880.0
2,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.809320,144.991833,100,NaN,NaN,3067,700.0,3000.0
3,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.808998,144.992535,55,NaN,NaN,3067,385.0,1650.0
4,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.808703,144.992925,90,NaN,NaN,3067,630.0,2700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.804382,144.996753,138,831.0,3461.0,3067,831.0,3461.0
96,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.803140,144.999114,46,300.0,NaN,3067,300.0,1380.0
97,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.801900,144.999080,101,NaN,NaN,3067,707.0,3030.0
98,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-03-12 00:00:00.000000,2.5,3067.0,...,VIC,3067.0,-37.802802,144.997031,150,NaN,NaN,3067,1050.0,4500.0


In [67]:
melb_df.groupby('Regionname').agg(
    total=('Price', 'count'),
    precio_promedio=('Price', 'mean'),
    precio_median=('Price', 'median'),
    precio_max=('Price', 'max'),
    precio_min=('Price', 'min')
).reset_index().sort_values('total', ascending=False)

,Regionname,total,precio_promedio,precio_median,precio_max,precio_min
5,Southern Metropolitan,4695,1.372963e+06,1250000.0,8000000.0,131000.0
2,Northern Metropolitan,3890,8.981711e+05,806250.0,4525000.0,145000.0
6,Western Metropolitan,2948,8.664205e+05,793000.0,3900000.0,85000.0
0,Eastern Metropolitan,1471,1.104080e+06,1010000.0,4000000.0,288000.0
4,South-Eastern Metropolitan,450,9.229438e+05,850000.0,9000000.0,266000.0
1,Eastern Victoria,53,6.999808e+05,670000.0,1085000.0,400000.0
3,Northern Victoria,41,5.948293e+05,540000.0,1355000.0,330000.0
7,Western Victoria,32,3.975234e+05,400000.0,710000.0,280500.0


In [68]:
melb_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount', 'Postcode_int'],
      dtype='object')

## Ejercicio 2 - Pandas:

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar explicitamente las columnas seleccionadas y las que no lo fueron.
  1. Valores faltantes: ¿Qué porcentaje de filas tienen al menos un valor faltante?
  2.  Mostrar la dispersión o distribución de las columnas seleccionadas.
 3.  Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.
 4. Mostrar visualmente los valores extremos que eliminás



2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico.
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana (porque no la media?) o el mínimo.
  2. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  3. Mostrar un gráfico zipcode vs airbnb_price_median.
  4. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.
  5. Si las coordenadas geoespaciales estuvieran disponibles, como las usarian?

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

¿Qué cosas no están en los datos que te gustaría tener para predecir mejor el precio de una propiedad?

In [ ]:
#1 -Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar explicitamente las columnas seleccionadas y las que no lo fueron.

#Vamos a utilizar la columnas Price, Rooms, Bedroom2 y bathroom ya que tienen una buena correlacion con el precio de las propiedades
correlaciones = pd.read_sql(queryJoin, con=engine).select_dtypes(include=[np.number]).corr().abs()
correlaciones['Price'].sort_values(ascending=False)

2025-05-20 21:00:17,056 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-20 21:00:17,058 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
select m.*, a.*
from melb m 
left join airbnb a 
    on m.Postcode_int = a.zipcode_int
LIMIT 100
")
2025-05-20 21:00:17,059 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 21:00:17,064 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
select m.*, a.*
from melb m 
left join airbnb a 
    on m.Postcode_int = a.zipcode_int
LIMIT 100
")
2025-05-20 21:00:17,067 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 21:00:17,069 INFO sqlalchemy.engine.Engine 
select m.*, a.*
from melb m 
left join airbnb a 
    on m.Postcode_int = a.zipcode_int
LIMIT 100

2025-05-20 21:00:17,070 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-20 21:00:17,428 INFO sqlalchemy.engine.Engine ROLLBACK


Rooms                     NaN
Price                     NaN
Distance                  NaN
Postcode                  NaN
Bedroom2                  NaN
Bathroom                  NaN
Car                       NaN
Landsize                  NaN
Lattitude                 NaN
Longtitude                NaN
Propertycount             NaN
Postcode_int              NaN
zipcode                   NaN
latitude                  NaN
longitude                 NaN
price                     NaN
weekly_price              NaN
monthly_price             NaN
zipcode_int               NaN
weekly_price_calculate    NaN
monthly_price_calculate   NaN
Name: Price, dtype: float64

In [ ]:
# Valores faltantes: ¿Qué porcentaje de filas tienen al menos un valor faltante?
rows = pandas.read_sql(queryJoin, con=engine)

totalValorFaltante = 0

for idx, fila in rows.iterrows():
    if fila.isnull().any():
        totalValorFaltante += 1
        continue

print(f"\nTotal de filas con al menos un valor faltante en su columna: {totalValorFaltante / len(rows) * 100:.2f}%")

In [ ]:
columnasSeleccionadas = ['Price', 'Rooms', 'Bedroom2', 'Bathroom']
result = pandas.read_sql(queryJoin, con=engine)[columnasSeleccionadas]

plt.figure(figsize=(8, 6))
plt.scatter(result['Rooms'], result['Price'], alpha=0.5)
plt.xlabel('Rooms')
plt.ylabel('Price')
plt.title('Dispersión: Rooms vs Price')
plt.show()



In [ ]:
result.sort_values(by='Price', ascending=True)


In [ ]:
seaborn.boxplot(x='Bathroom', y='Price', data=result)
plt.title('Dispersion: Rooms vs Price')
plt.show()

In [ ]:
#Eliminar valores atípicos de propiedades con valores mayor a 6 millone

pandas.set_option('display.float_format', lambda x: '%.2f' % x)
quantile=  resultJoin['Price'].quantile(0.90)


queryJoin = "select m.*, a.weekly_price, a.monthly_price from melb m left join airbnb a on m.postcode = a.zipcode where price < "+ str(quantile)    

resultJoin = pandas.read_sql(queryJoin, con=engine)

ax = seaborn.boxenplot(x='Rooms', y='Price', data=resultJoin)
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f'))

plt.title('Dispersion sin valores extremos: Rooms vs Price')
plt.show()

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)

In [3]:
pip install -U decouple


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import datetime as dt
import requests
import io
from sqlalchemy import create_engine, text
import os
from decouple import config
import logging

ImportError: cannot import name 'config' from 'decouple' (/Users/peluso/Metodos_numericos_nbodys/simulacao_doutorado/env_graph/lib/python3.10/site-packages/decouple/__init__.py)

In [2]:
# Cuando nos queremos conectar a una Base de Datos productiva normalmente necesitamos pasar credenciales para acceder.
# Estas credenciales NO deben ser escritas en archivos compartidos subidos a github, sino mas bien en archivos "privados".
# Una buena practica para manejar credenciales es en archivos ".env" que solo quedan registrados en su computadora local.
# La libreria "python decouple" permite leer estos archivos de configuracion .env y manejarlo como variables.
DB_USER = config('DB_USER')
DB_PASSWORD = config('DB_PASSWORD')
DB_HOST = config('DB_HOST')
DB_PORT = config('DB_PORT')
# En lugar de usar prints para ver el avance a medida que va corriendo el script se utilizan los logs.
# Los logs basicamente son registros que se van dejando para saber el codigo que ha sido ejecutado.
# Es decision arbitraria del programador decidir que desea registrar en los logs.
# En python se utiliza la libreria logging https://docs.python.org/3/library/logging.html#logging-levels
# La libreria permite definir niveles de logs (ERROR, DEBUG, INFO, etc). Segun la criticidad del error.
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')


NameError: name 'config' is not defined

In [ ]:
def extract(url,interesting_cols):
  # The extract process could be complex including some SQL queries
    if len(interesting_cols) ==0:
        df = pd.read_csv(url)
        logger.info('read_csv exitoso')
    else:
        df = pd.read_csv(url,usecols=interesting_cols)
        logger.info('read_csv exitoso con columnas interesantes')
    return df

In [10]:
def connection_db(database_name):
    '''Connect to DB using SQLAlchemy methods. Returns an engine created and connected'''
    # ejemplo de conexion a PostgreSQL utilizando SQLalchemy
    #engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/survey".format(),echo=False, client_encoding='utf8')
    try:
        engine = create_engine(f"sqlite:///{database_name}", echo=True)
        logger.info('Conexion exitosa a la base de datos')
        return engine

    except ValueError as e:
        logger.error(e)

In [ ]:
def transform(df,database_table_name):
  '''Transform the data. In this case, we are going to use the same dataframe'''
  # En este caso no se realizan transformaciones
  # pero se pueden realizar transformaciones como eliminar columnas, renombrar columnas, etc.
  if database_table_name == 'melb'
    melb_df['Postcode'] = pd.to_numeric(melb_df.Postcode, errors='coerce')
    melb_df['Postcode_int'] = melb_df.Postcode.fillna(0).astype('int') # Convertimos la columna Postcode a int y renombramos la columna para no perder la original
    melb_df['Date'] = pd.to_datetime(melb_df['Date'], errors='coerce') # Convertimos la columna Date a datetime
    melb_df['Bedroom2'] = melb_df['Bedroom2'].fillna(0).astype('int') # Convertimos la columna Bedroom2 a int
    melb_df['Bathroom'] = melb_df['Bathroom'].fillna(0).astype('int') # Convertimos la columna Bathroom a int
    melb_df['Car'] = melb_df['Car'].fillna(0).astype('int') # Convertimos la columna Car a int
  else:
    # Working data for the analysis
    airbnb_df['zipcode'] = pd.to_numeric(airbnb_df.zipcode, errors='coerce')
    airbnb_df['zipcode_int'] = airbnb_df.zipcode.fillna(0).astype('int') # Convertimos la columna zipcode a int y renombramos la columna para no perder la original
    airbnb_df['weekly_price_calculate'] = airbnb_df.apply(lambda row: row['price']*7. if pd.isnull(row['weekly_price']) else row['weekly_price'], axis=1) # Realizamos el calculo de weekly_price considerando price*7dias
    airbnb_df['monthly_price_calculate'] = airbnb_df.apply(lambda row: row['price']*30. if pd.isnull(row['monthly_price']) else row['monthly_price'], axis=1) # Realizamos el calculo de weekly_price considerando price*30dias



In [ ]:
def save_database(df,table_name):
  '''Load the data into the database'''
  try:
      # Create a table 'melb' in the SQLite database
      df.to_sql(table_name, con=engine, if_exists='replace', index=False)
      logger.info('Carga exitosa de la tabla en la base de datos')
  except ValueError as e:
      logger.error(e)
      logger.error('Error en la carga de la tabla en la base de datos')
    

In [ ]:
# la funcion main es muy utilizada en scripts python cuando tenemos archivos .py por ejemplo etl.py
# al tener la funcion main pueden correr desde la terminal python etl.py y va a ejecutar lo definido en la funcion main
def main():

    logger.info('Comienza la extraccion')
    logger.info('Conectar a la base de datos')
    engine = connection_db('melbourne.db')
    logger.info('Extract data from url melbourne dataset')
    df = extract('http://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv', [])
    logger.info('Transform data')
    transform(df,'melb')
    logger.info('Save data into database')
    save_database(df,engine,'melb')
    logger.info('Extract data from url airbnb dataset')
    df = extract('http://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv', ['description', 'neighborhood_overview','street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode','price', 'weekly_price', 'monthly_price','latitude', 'longitude'])
    logger.info('Transform data')
    transform(df,'airbnb')
    logger.info('Save data into database')
    save_database(df,'airbnb')

if __name__ == "__main__":
    logger.info('ETL Process Initialized')
    main()

In [ ]:
# ejemplo de DAG en Airflow for melb ETL dataset
from airflow import DAG
with DAG(
    'dag_melb',
    default_args=default_args,
    description='DAG ',
    schedule_interval=timedelta(hours=1),
    start_date=datetime(2025, 6, 26),
) as dag:
    connection = PythonOperator(task_id='engine',
                                python_callable=connection_db(),
                                op_kwargs={database_name: 'melbourne.db'},)
    extraction = PythonOperator(task_id='engine',
                                python_callable=extract(),
                                op_kwargs={url:'http://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv',interesting_cols:[]},)
    transformation = PythonOperator(task_id='transformation',
                                    python_callable=transform
                                    op_kwargs={database_table_name: 'melb')
    save_database = PythonOperator(task_id='save_database',
                                 python_callable=save_database,
                                 op_kwargs={table_name:'melb'},)
    connection >> extraction >> transformation >> save_database

# ejemplo de DAG en Airflow for airbnb ETL dataset
from airflow import DAG
with DAG(
    'dag_airbnb',
    default_args=default_args,
    description='DAG ',
    schedule_interval=timedelta(hours=1),
    start_date=datetime(2025, 6, 26),
) as dag:
    connection = PythonOperator(task_id='engine',
                                python_callable=connection_db(),
                                op_kwargs={database_name: 'melbourne.db'},)
    extraction = PythonOperator(task_id='engine',
                                python_callable=extract(),
                                op_kwargs={
                                    url:'http://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
                                    interesting_cols:['description', 'neighborhood_overview','street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode','price', 'weekly_price', 'monthly_price','latitude', 'longitude']},)
    transformation = PythonOperator(task_id='transformation',
                                    python_callable=transform
                                    op_kwargs={database_table_name: 'airbnb')
    save_database = PythonOperator(task_id='save_database',
                                 python_callable=save_database,
                                 op_kwargs={table_name:'airbnb'},)
    connection >> extraction >> transformation >> save_database